In [1]:
import pandas as pd
import tensorflow as tf

  # Cargar el dataset
data = pd.read_csv('FIS\dataset.csv')

  # Mostrar las primeras filas del dataset
print(data.head())

          P         D          L          R P_fuzzy_set  P_membership  \
0 -0.344708  0.267949  50.949905  24.525778       lejos      0.861770   
1 -0.512704 -0.725829  90.972135 -53.639732       lejos      0.812159   
2  0.388981 -0.031825 -49.601000  63.894966       cerca      0.972454   
3 -0.918004 -0.325898  92.482375 -50.164266   muy lejos      0.863339   
4  0.046981 -0.488745  51.090865  -2.041475          ok      0.882547   

   D_fuzzy_set  D_membership                     L_fuzzy_set  L_membership  \
0  acercandose      1.000000      medio rápido hacia delante      0.905009   
1   alejandose      1.000000        muy rápido hacia delante      1.000000   
2   sin cambio      0.840877  más o menos rápido hacia atrás      0.759975   
3   alejandose      1.000000        muy rápido hacia delante      1.000000   
4   alejandose      1.000000      medio rápido hacia delante      0.890913   

                      R_fuzzy_set  R_membership  \
0         despacito hacia delante      0.

In [2]:
X = data[['P','D','L','R']].values
y = data['CAT'].values

In [3]:
from sklearn.preprocessing import StandardScaler

# Ajusta las columnas numéricas con StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(y)
num_decoder_tokens = len(tokenizer.word_index) + 1
y_tokenized = tokenizer.texts_to_sequences(y)

In [5]:
max_len = max(len(s) for s in y_tokenized)

In [6]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_tokenized, test_size=0.2, random_state=42)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ajustar la longitud de las secuencias de frases en y_train a max_len
y_train = pad_sequences(y_train, maxlen=num_decoder_tokens, padding='post', truncating='post')

# Ajustar la longitud de las secuencias de frases en y_val a max_len
y_val = pad_sequences(y_val, maxlen=num_decoder_tokens, padding='post', truncating='post')

# Ajustar la longitud de las secuencias de frases en y_test a max_len
y_test = pad_sequences(y_test, maxlen=num_decoder_tokens, padding='post', truncating='post')

In [8]:
print('tamaño X train', X_train.shape)
print('\ntamaño y train', y_train.shape)
print('\ntamaño X val', X_val.shape)
print('\ntamaño y val', y_val.shape)
print('\ntamaño X test', X_test.shape)
print('\ntamaño y test', y_test.shape)


tamaño X train (5999, 4)

tamaño y train (5999, 41)

tamaño X val (2000, 4)

tamaño y val (2000, 41)

tamaño X test (2000, 4)

tamaño y test (2000, 41)


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Definir dimension de entrada para el encoder
encoder_inputs = Input(shape=(1, 4))

# Capa LSTM en el encoder
encoder_lstm = LSTM(256, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)

# Se descartan las salidas del encoder y solo se toman los estados
encoder_states = [state_h, state_c]

In [10]:
# Definir dimension de entrada para el decoder
decoder_inputs = Input(shape=(None,))

# Capa de embedding en el decoder
decoder_embedding = Embedding(num_decoder_tokens, output_dim=100)
decoder_inputs_embedded = decoder_embedding(decoder_inputs)

# Capa LSTM en el decoder, con los estados del encoder como inicialización
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_embedded, initial_state=encoder_states)

# Capa densa con activación softmax en el output
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Definir modelo encoder-decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilar el modelo
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [40]:
type(y_train)

numpy.ndarray

In [12]:
# Entrenar el modelo
model.fit([X_train.reshape(5999,1,4), y_train], y_train.reshape(y_train.shape[0], y_train.shape[1], 1), batch_size=128, epochs=15, validation_data=([X_val.reshape(X_val.shape[0],1,4), y_val], y_val.reshape(y_val.shape[0], y_val.shape[1], 1)))

Epoch 1/15
47/47 [==============================] - 19s 307ms/step - loss: 2.7771 - val_loss: 2.4274
Epoch 2/15
47/47 [==============================] - 13s 280ms/step - loss: 2.1234 - val_loss: 1.7266
Epoch 3/15
47/47 [==============================] - 13s 276ms/step - loss: 1.3718 - val_loss: 1.0745
Epoch 4/15
47/47 [==============================] - 13s 277ms/step - loss: 0.8733 - val_loss: 0.6861
Epoch 5/15
47/47 [==============================] - 13s 280ms/step - loss: 0.4824 - val_loss: 0.3068
Epoch 6/15
47/47 [==============================] - 13s 277ms/step - loss: 0.2093 - val_loss: 0.1448
Epoch 7/15
47/47 [==============================] - 13s 277ms/step - loss: 0.0793 - val_loss: 0.0434
Epoch 8/15
47/47 [==============================] - 14s 288ms/step - loss: 0.0257 - val_loss: 0.0156
Epoch 9/15
47/47 [==============================] - 14s 294ms/step - loss: 0.0110 - val_loss: 0.0080
Epoch 10/15
47/47 [==============================] - 14s 289ms/step - loss: 0.0064 - val_lo

In [13]:
y_test[0]

array([11, 12, 16, 26,  8,  9, 17, 21, 23,  6, 31, 25, 23,  6, 32, 19, 18,
        2,  3,  4, 13, 20,  5, 10,  7,  2,  3,  4, 14, 20,  5, 10,  0,  0,
        0,  0,  0,  0,  0,  0,  0])

In [14]:
model.evaluate([X_test.reshape(X_test.shape[0],1,4),y_test[:,:-1]])

63/63 [==============================] - 1s 2ms/step - loss: 0.0000e+00


0.0

In [15]:
import numpy as np

In [16]:
preds = model.predict([X_test.reshape(X_test.shape[0], 1, 4), y_test[:,:-1]])

# Convertir las predicciones de índices a palabras
preds = np.argmax(preds, axis=-1)


63/63 [==============================] - 5s 55ms/step


In [17]:
preds

array([[11, 12, 16, ...,  0,  0,  0],
       [11, 12,  8, ...,  0,  0,  0],
       [11, 12,  8, ...,  0,  0,  0],
       ...,
       [11, 12, 28, ...,  0,  0,  0],
       [11, 12, 16, ...,  0,  0,  0],
       [11, 12, 16, ...,  0,  0,  0]], dtype=int64)

In [18]:

def predictions(preds):
    preds_words = []
    for i in range(preds.shape[0]):
        preds_row = preds[i]
        preds_row_words = []
        for idx in preds_row:
            if idx in tokenizer.index_word:
                preds_row_words.append(tokenizer.index_word[idx])
        preds_words.append(preds_row_words)

    # Imprimir las primeras 10 predicciones
    for i in range(10):
        print(preds_words[i])

    return preds_words

In [19]:
frasesnlg = predictions(preds)

['cuando', 'estés', 'muy', 'cerca', 'a', 'la', 'pared', 'pero', 'ni', 'te', 'acercas', 'más', 'ni', 'te', 'alejas', 'de', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'despacito', 'hacia', 'atrás', 'y', 'gira', 'tu', 'rueda', 'derecha', 'despacito', 'hacia', 'atrás']
['cuando', 'estés', 'a', 'la', 'distancia', 'requerida', 'y', 'ni', 'te', 'alejas', 'ni', 'te', 'acercas', 'a', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'despacito', 'hacia', 'delante', 'y', 'gira', 'tu', 'rueda', 'derecha', 'despacito', 'hacia', 'delante']
['cuando', 'estés', 'a', 'la', 'distancia', 'requerida', 'y', 'ni', 'te', 'alejas', 'ni', 'te', 'acercas', 'a', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'despacito', 'hacia', 'delante', 'y', 'gira', 'tu', 'rueda', 'derecha', 'despacito', 'hacia', 'delante']
['cuando', 'estés', 'cerca', 'a', 'la', 'pared', 'pero', 'ni', 'te', 'acercas', 'más', 'ni', 'te', 'alejas', 'de', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'bastante', 'rápido', 'hacia', 'atrás', 'y', 'gira',

In [20]:
x_values = scaler.inverse_transform(X_test)

In [21]:
x_values[0]

array([  0.97207347,  -0.0330287 , -27.74293442, -12.93248209])

In [22]:
preds_words = []
for i in range(y_test.shape[0]):
    preds_row = y_test[i]
    preds_row_words = []
    for idx in preds_row:
        if idx in tokenizer.index_word:
            preds_row_words.append(tokenizer.index_word[idx])
    preds_words.append(preds_row_words)
for i in range(10):
    print(preds_words[i])

['cuando', 'estés', 'muy', 'cerca', 'a', 'la', 'pared', 'pero', 'ni', 'te', 'acercas', 'más', 'ni', 'te', 'alejas', 'de', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'despacito', 'hacia', 'atrás', 'y', 'gira', 'tu', 'rueda', 'derecha', 'despacito', 'hacia', 'atrás']
['cuando', 'estés', 'a', 'la', 'distancia', 'requerida', 'y', 'ni', 'te', 'alejas', 'ni', 'te', 'acercas', 'a', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'despacito', 'hacia', 'delante', 'y', 'gira', 'tu', 'rueda', 'derecha', 'despacito', 'hacia', 'delante']
['cuando', 'estés', 'a', 'la', 'distancia', 'requerida', 'y', 'ni', 'te', 'alejas', 'ni', 'te', 'acercas', 'a', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'despacito', 'hacia', 'delante', 'y', 'gira', 'tu', 'rueda', 'derecha', 'despacito', 'hacia', 'delante']
['cuando', 'estés', 'cerca', 'a', 'la', 'pared', 'pero', 'ni', 'te', 'acercas', 'más', 'ni', 'te', 'alejas', 'de', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'bastante', 'rápido', 'hacia', 'atrás', 'y', 'gira',

In [23]:
pdlr = np.array([[-0.04182976,0.393413729,-3.725535952,54.77127996]])

In [30]:
pdlrsalida =np.zeros(1,41)

TypeError: Cannot interpret '41' as a data type

In [28]:
y_test.shape

(2000, 41)

In [31]:
prueba = model.predict([pdlr.reshape(1,1,4),np.zeros((1,41))])

1/1 [==============================] - 2s 2s/step


In [36]:
prueba = np.argmax(prueba, axis=-1)

In [38]:
prueba

array([[8, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

In [37]:
predictions(prueba)

['a', 'la']


IndexError: list index out of range